In [1]:
import pandas as pd
import linearmodels
import statsmodels.formula.api as smf
import statsmodels.api as sm
import numpy as np

C:\Users\Jeremy\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
df = pd.read_stata('SeatBelts.dta')

In [3]:
df.head()

,state,year,fips,vmt,fatalityrate,sb_useage,speed65,speed70,drinkage21,ba08,income,age,primary,secondary
0,AK,1983,2,3358.0,0.044669,NaN,0.0,0.0,1.0,0.0,17973.0,28.234966,0.0,0.0
1,AK,1984,2,3589.0,0.037336,NaN,0.0,0.0,1.0,0.0,18093.0,28.343542,0.0,0.0
2,AK,1985,2,3840.0,0.033073,NaN,0.0,0.0,1.0,0.0,18925.0,28.372816,0.0,0.0
3,AK,1986,2,4008.0,0.025200,NaN,0.0,0.0,1.0,0.0,18466.0,28.396652,0.0,0.0
4,AK,1987,2,3900.0,0.019487,NaN,0.0,0.0,1.0,0.0,18021.0,28.453251,0.0,0.0


In [4]:
df['lincome'] = np.log(df.income)

#### Q1.

In [5]:
formula = 'fatalityrate ~ + sb_useage + speed65 + speed70 + ba08 + drinkage21 + lincome + age'
reg = smf.ols(formula=formula, data=df).fit()
reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           fatalityrate   R-squared:                       0.549
Model:                            OLS   Adj. R-squared:                  0.544
Method:                 Least Squares   F-statistic:                     95.41
Date:                Sat, 19 May 2018   Prob (F-statistic):           1.26e-90
Time:                        23:12:37   Log-Likelihood:                 2375.7
No. Observations:                 556   AIC:                            -4735.
Df Residuals:                     548   BIC:                            -4701.
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.1965      0.008     23.902      0.000       0.180       0.213
sb_useage      0.0041      0.001      3.346      0.001       0.002       0.006
speed65        0.0001      0.000      0.367      0.714      -0.001       0.001
speed70        0.0024      0.001      4.704      0.000       0.001       0.003
ba08          -0.0019      0.000     -4.327      0.000      -0.003      -0.001
drinkage21  7.988e-05      0.001      0.091      0.927      -0.002       0.002
lincome       -0.0181      0.001    -19.487      0.000      -0.020      -0.016
age         -7.22e-06      0.000     -0.066      0.947      -0.000       0.000
==============================================================================
Omnibus:                       42.667   Durbin-Watson:                   0.456
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               59.466
Skew:                           0.592   Prob(JB):                     1.22e-13
Kurtosis:                       4.079   Cond. No.                     2.12e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.12e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [6]:
# # Interesting that removing the constant flips the sign of sb_usage
# The p-values are low for each case (with and without the intercept)
# formula = 'fatalityrate ~ -1 + sb_useage + speed65 + speed70 + ba08 + drinkage21 + lincome + age'
# reg = smf.ols(formula=formula, data=df).fit()
# reg.summary()

#### Q2.

In [7]:
df_indexed = df.set_index(['state', 'year'])
df_indexed = df_indexed[['fatalityrate', 'sb_useage', 'speed65', 'speed70', 
                         'ba08', 'drinkage21', 'lincome', 'age']].dropna()

In [8]:
df_indexed.head()

fatalityrate  sb_useage  speed65  speed70  ba08  drinkage21  \
state year                                                                
AK    1990      0.024629       0.45      0.0      0.0   0.0         1.0   
      1991      0.025118       0.66      0.0      0.0   0.0         1.0   
      1992      0.028118       0.66      1.0      0.0   0.0         1.0   
      1993      0.030117       0.69      1.0      0.0   0.0         1.0   
      1994      0.020482       0.69      1.0      0.0   0.0         1.0   

              lincome        age  
state year                        
AK    1990   9.955748  29.586285  
      1991   9.975622  29.827711  
      1992  10.002110  30.210697  
      1993  10.030604  30.464386  
      1994  10.061217  30.756571

In [9]:
formula = 'fatalityrate ~ sb_useage + speed65 + speed70 + ba08 + \
    drinkage21 + lincome + age + EntityEffects'
reg = linearmodels.PanelOLS.from_formula(formula=formula, data=df_indexed).fit()
reg

Dep. Variable:,fatalityrate,R-squared:,0.6868
Estimator:,PanelOLS,R-squared (Between):,-35.138
No. Observations:,556,R-squared (Within):,0.6868
Date:,"Sat, May 19 2018",R-squared (Overall):,-34.226
Time:,23:12:37,Log-likelihood,2759.6
Cov. Estimator:,Unadjusted,,
,,F-statistic:,156.00
Entities:,51,P-value,0.0000
Avg Obs:,10.902,Distribution:,"F(7,498)"
Min Obs:,8.0000,,
Max Obs:,15.000,F-statistic (robust):,156.00


In [10]:
# Notice that the estimates don't change (reporting the average of the
# fixed effects?)
formula = 'fatalityrate ~ 1 + sb_useage + speed65 + speed70 + ba08 + \
    drinkage21 + lincome + age + EntityEffects'
reg = linearmodels.PanelOLS.from_formula(formula=formula, data=df_indexed).fit()
reg

Dep. Variable:,fatalityrate,R-squared:,0.6868
Estimator:,PanelOLS,R-squared (Between):,0.1502
No. Observations:,556,R-squared (Within):,0.6868
Date:,"Sat, May 19 2018",R-squared (Overall):,0.3806
Time:,23:12:37,Log-likelihood,2759.6
Cov. Estimator:,Unadjusted,,
,,F-statistic:,156.00
Entities:,51,P-value,0.0000
Avg Obs:,10.902,Distribution:,"F(7,498)"
Min Obs:,8.0000,,
Max Obs:,15.000,F-statistic (robust):,156.00


#### Q3.

In [11]:
formula = 'fatalityrate ~ sb_useage + speed65 + speed70 + ba08 + \
    drinkage21 + lincome + age + EntityEffects + TimeEffects'
reg = linearmodels.PanelOLS.from_formula(formula=formula, data=df_indexed).fit()
reg

Dep. Variable:,fatalityrate,R-squared:,0.0828
Estimator:,PanelOLS,R-squared (Between):,-16.879
No. Observations:,556,R-squared (Within):,-0.5899
Date:,"Sat, May 19 2018",R-squared (Overall):,-16.479
Time:,23:12:38,Log-likelihood,2823.0
Cov. Estimator:,Unadjusted,,
,,F-statistic:,6.2388
Entities:,51,P-value,0.0000
Avg Obs:,10.902,Distribution:,"F(7,484)"
Min Obs:,8.0000,,
Max Obs:,15.000,F-statistic (robust):,6.2388
